## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [3]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]


In [4]:
# Create a list of at least 10 column names to use as X data
X_columns_to_include = [column_name for column_name in attrition_df.columns if column_name not in ('Attrition', 'Department')]

# Create X_df using your selected columns
X_df = attrition_df[X_columns_to_include]

# Show the data types for X_df
X_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 25 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Age                       1470 non-null   int64 
 1   BusinessTravel            1470 non-null   object
 2   DistanceFromHome          1470 non-null   int64 
 3   Education                 1470 non-null   int64 
 4   EducationField            1470 non-null   object
 5   EnvironmentSatisfaction   1470 non-null   int64 
 6   HourlyRate                1470 non-null   int64 
 7   JobInvolvement            1470 non-null   int64 
 8   JobLevel                  1470 non-null   int64 
 9   JobRole                   1470 non-null   object
 10  JobSatisfaction           1470 non-null   int64 
 11  MaritalStatus             1470 non-null   object
 12  NumCompaniesWorked        1470 non-null   int64 
 13  OverTime                  1470 non-null   object
 14  PercentSalaryHike       

In [5]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df)

In [6]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary

# Filtering out string columns
X_string_columns = X_df.select_dtypes(include=['object']).columns

# Create training string df
X_train_string = X_train[X_string_columns]

# Create testing string df
X_test_string = X_test[X_string_columns]

# Check unique values in each column
print("X string column unique value counts")
print(X_train_string.nunique())
print(f"Expected number of columns after encoding: {X_test_string.nunique().sum()}")

X string column unique value counts
BusinessTravel    3
EducationField    6
JobRole           9
MaritalStatus     3
OverTime          2
dtype: int64
Expected number of columns after encoding: 23


In [7]:
# Import OneHotEncoder
from sklearn.preprocessing import OneHotEncoder

# Instantiate OHE
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# Fit and transform string training and testing data
X_train_string_encoded = ohe.fit_transform(X_train_string)
X_test_string_encoded = ohe.transform(X_test_string)

# Get column names
encoded_column_names = ohe.get_feature_names_out(X_string_columns)

# Create encoded dfs
X_train_string_encoded_df = pd.DataFrame(X_train_string_encoded, columns=encoded_column_names)
X_test_string_encoded_df = pd.DataFrame(X_test_string_encoded, columns=encoded_column_names)

# Drop redundant binary column 'OverTime_No'
X_train_string_encoded_df = X_train_string_encoded_df.drop('OverTime_No', axis=1)
X_test_string_encoded_df = X_test_string_encoded_df.drop('OverTime_No', axis=1)

# Check encoded dfs
display(X_train_string_encoded_df.head())
X_test_string_encoded_df.head()


,BusinessTravel_Non-Travel,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,EducationField_Human Resources,EducationField_Life Sciences,EducationField_Marketing,EducationField_Medical,EducationField_Other,EducationField_Technical Degree,JobRole_Healthcare Representative,...,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,OverTime_Yes
0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
3,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


,BusinessTravel_Non-Travel,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,EducationField_Human Resources,EducationField_Life Sciences,EducationField_Marketing,EducationField_Medical,EducationField_Other,EducationField_Technical Degree,JobRole_Healthcare Representative,...,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,OverTime_Yes
0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [8]:
# Isolate numeric columns for training and testing data
X_train_numeric = X_train.drop(columns=X_string_columns).reset_index(drop=True)
X_test_numeric = X_test.drop(columns=X_string_columns).reset_index(drop=True)

# Merge encoded df with numeric df
X_train_encoded = pd.concat([X_train_numeric, X_train_string_encoded_df], axis=1)
X_test_encoded = pd.concat([X_test_numeric, X_test_string_encoded_df], axis=1)
# Check merged dfs
display(X_train_encoded.head())
X_test_encoded.head()

,Age,DistanceFromHome,Education,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,JobSatisfaction,NumCompaniesWorked,PercentSalaryHike,...,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,OverTime_Yes
0,39,9,2,1,87,3,2,1,1,25,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,40,2,3,3,98,3,2,4,1,13,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,35,8,2,3,78,2,3,4,4,11,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
3,55,26,5,3,60,2,5,4,1,21,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,26,1,1,1,66,2,1,3,1,13,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


,Age,DistanceFromHome,Education,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,JobSatisfaction,NumCompaniesWorked,PercentSalaryHike,...,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,OverTime_Yes
0,26,25,1,3,61,3,1,3,1,23,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,38,1,3,4,90,3,2,4,7,15,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,51,6,3,1,51,3,5,3,7,13,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,36,5,2,4,62,3,2,2,8,16,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,28,26,3,3,48,2,2,1,1,11,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [9]:
# Create a StandardScaler
scaler = StandardScaler()

# Scale the training and testing data
X_train_encoded_scaled = scaler.fit_transform(X_train_encoded)
X_test_encoded_scaled = scaler.transform(X_test_encoded)

# Get column names
scaled_column_names = scaler.get_feature_names_out(X_train_encoded.columns)

# Create dfs from scaled data
X_train_encoded_scaled_df = pd.DataFrame(X_train_encoded_scaled, columns=scaled_column_names)
X_test_encoded_scaled_df = pd.DataFrame(X_test_encoded_scaled, columns=scaled_column_names)

# Check dfs
display(X_train_encoded_scaled_df.head())
X_test_encoded_scaled_df.head()

,Age,DistanceFromHome,Education,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,JobSatisfaction,NumCompaniesWorked,PercentSalaryHike,...,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,OverTime_Yes
0,0.255426,-0.009544,-0.886776,-1.583496,1.051035,0.373087,-0.055578,-1.605835,-0.667763,2.689007,...,-0.277892,3.033761,-0.23355,-0.499433,-0.533275,-0.244163,-0.514988,1.087280,-0.705183,-0.615812
1,0.365464,-0.886040,0.079819,0.262102,1.590755,0.373087,-0.055578,1.134945,-0.667763,-0.604448,...,-0.277892,-0.329624,-0.23355,2.002271,-0.533275,-0.244163,-0.514988,-0.919726,1.418072,-0.615812
2,-0.184729,-0.134758,-0.886776,0.262102,0.609445,-1.054489,0.832052,1.134945,0.521050,-1.153357,...,-0.277892,-0.329624,-0.23355,-0.499433,1.875205,-0.244163,-0.514988,1.087280,-0.705183,1.623873
3,2.016045,2.119087,2.013008,0.262102,-0.273734,-1.054489,2.607311,1.134945,-0.667763,1.591189,...,3.598523,-0.329624,-0.23355,-0.499433,-0.533275,-0.244163,-0.514988,1.087280,-0.705183,-0.615812
4,-1.175078,-1.011253,-1.853371,-1.583496,0.020659,-1.054489,-0.943207,0.221352,-0.667763,-0.604448,...,-0.277892,-0.329624,-0.23355,2.002271,-0.533275,-0.244163,1.941794,-0.919726,-0.705183,-0.615812


,Age,DistanceFromHome,Education,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,JobSatisfaction,NumCompaniesWorked,PercentSalaryHike,...,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,OverTime_Yes
0,-1.175078,1.993873,-1.853371,0.262102,-0.224668,0.373087,-0.943207,0.221352,-0.667763,2.140098,...,-0.277892,-0.329624,-0.233550,-0.499433,-0.533275,-0.244163,-0.514988,1.087280,-0.705183,-0.615812
1,0.145387,-1.011253,0.079819,1.184901,1.198231,0.373087,-0.055578,1.134945,1.709863,-0.055538,...,-0.277892,-0.329624,-0.233550,2.002271,-0.533275,-0.244163,-0.514988,1.087280,-0.705183,-0.615812
2,1.575890,-0.385185,0.079819,-1.583496,-0.715323,0.373087,2.607311,0.221352,1.709863,-0.604448,...,-0.277892,-0.329624,4.281744,-0.499433,-0.533275,-0.244163,-0.514988,-0.919726,1.418072,-0.615812
3,-0.074691,-0.510399,-0.886776,1.184901,-0.175603,0.373087,-0.055578,-0.692242,2.106134,0.218916,...,-0.277892,-0.329624,-0.233550,-0.499433,-0.533275,-0.244163,-0.514988,-0.919726,1.418072,-0.615812
4,-0.955000,2.119087,0.079819,0.262102,-0.862520,-1.054489,-0.055578,-1.605835,-0.667763,-1.153357,...,-0.277892,-0.329624,-0.233550,-0.499433,1.875205,-0.244163,-0.514988,1.087280,-0.705183,-0.615812


In [10]:
# Check y_df information and unique value counts
y_df.info()
display(y_df.nunique())
print(f"Expected number of columns after encoding: {y_df.nunique().sum()}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Attrition   1470 non-null   object
 1   Department  1470 non-null   object
dtypes: object(2)
memory usage: 23.1+ KB


Attrition     2
Department    3
dtype: int64

Expected number of columns after encoding: 5


In [11]:
# Create a OneHotEncoder for the Department and Attrition column
y_ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# Fit the encoder to the training data, transform y_train and y_test
y_train_encoded = y_ohe.fit_transform(y_train)
y_test_encoded = y_ohe.transform(y_test)

# Get column names from y_df
y_df_column_names = y_ohe.get_feature_names_out(y_df.columns)

# Turn encoded data back into dataframes
y_train_encoded_df = pd.DataFrame(y_train_encoded, columns=y_df_column_names)
y_test_encoded_df = pd.DataFrame(y_test_encoded, columns=y_df_column_names)

# Drop redundant binary column 'Attrition No'
y_train_encoded_df = y_train_encoded_df.drop('Attrition_No', axis=1)
y_test_encoded_df = y_test_encoded_df.drop('Attrition_No', axis=1)

# Check encoded dfs
display(y_train_encoded_df.head())
y_test_encoded_df.head()

,Attrition_Yes,Department_Human Resources,Department_Research & Development,Department_Sales
0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,1.0
4,0.0,0.0,1.0,0.0


,Attrition_Yes,Department_Human Resources,Department_Research & Development,Department_Sales
0,0.0,1.0,0.0,0.0
1,0.0,0.0,1.0,0.0
2,0.0,0.0,1.0,0.0
3,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0


In [12]:
# Seperate into Attrition and Department dfs
# Create funtion to split df by keyword
def isolate_by_keyword(df, keyword):
    iso_columns = df.columns.str.contains(keyword, case=False)
    iso_df = df.loc[:, iso_columns]
    return iso_df

# Create dfs
y_train_attrition = isolate_by_keyword(y_train_encoded_df, 'attrition')
y_test_attrition = isolate_by_keyword(y_test_encoded_df, 'attrition')
y_train_department = isolate_by_keyword(y_train_encoded_df, 'department')
y_test_department = isolate_by_keyword(y_test_encoded_df, 'department')

# Check dfs
display(y_train_attrition.head())
display(y_test_department.head())


,Attrition_Yes
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


,Department_Human Resources,Department_Research & Development,Department_Sales
0,1.0,0.0,0.0
1,0.0,1.0,0.0
2,0.0,1.0,0.0
3,0.0,1.0,0.0
4,0.0,0.0,1.0


In [13]:
# Final preprocessed dfs are:

# X_train_encoded_scaled_df
# X_test_encoded_scaled_df

# y_train_attrition
# y_test_attrition

# y_train_department
# y_test_department

## Create, Compile, and Train the Model

In [14]:
# Find the number of columns in the X training data
X_col_count = len(X_train_encoded_scaled_df.columns)

# Create the input layer
input_layer = layers.Input(shape=(X_col_count), name='input_features')

# Create at least two shared layers
shared_layer1 = layers.Dense(32, activation='relu', name='shared_1')(input_layer)
shared_layer2 = layers.Dense(32, activation='relu', name='shared_2')(shared_layer1)

In [15]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_layer1 = layers.Dense(32, activation='relu', name='department_1')(shared_layer2)

# Create the output layer
department_output = layers.Dense(3, activation='softmax', name='department_output')(department_layer1)


In [16]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_layer1 = layers.Dense(32, activation='relu', name='attrition_1')(shared_layer2)

# Create the output layer
attrition_output = layers.Dense(1, activation='sigmoid', name='attrition_output')(attrition_layer1)


In [17]:
# Create the model
model = Model(inputs=input_layer, outputs=[department_output, attrition_output])

# Compile the model
model.compile(optimizer='adam',
              loss={'department_output': 'categorical_crossentropy', 'attrition_output': 'binary_crossentropy'},
              metrics={'department_output': 'accuracy', 'attrition_output': 'accuracy'})

# Summarize the model
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_features (InputLayer  [(None, 42)]                 0         []                            
 )                                                                                                
                                                                                                  
 shared_1 (Dense)            (None, 32)                   1376      ['input_features[0][0]']      
                                                                                                  
 shared_2 (Dense)            (None, 32)                   1056      ['shared_1[0][0]']            
                                                                                                  
 department_1 (Dense)        (None, 32)                   1056      ['shared_2[0][0]']       

In [18]:
# Train the model
model.fit(
    X_train_encoded_scaled_df,
    {'department_output': y_train_department, 'attrition_output': y_train_attrition},
    epochs=10
)


Epoch 1/10


35/35 [==============================] - 1s 1ms/step - loss: 1.5421 - department_output_loss: 0.9505 - attrition_output_loss: 0.5916 - department_output_accuracy: 0.5889 - attrition_output_accuracy: 0.7060
Epoch 2/10
35/35 [==============================] - 0s 1ms/step - loss: 1.0961 - department_output_loss: 0.6599 - attrition_output_loss: 0.4362 - department_output_accuracy: 0.7359 - attrition_output_accuracy: 0.8385
Epoch 3/10
35/35 [==============================] - 0s 953us/step - loss: 0.8661 - department_output_loss: 0.4655 - attrition_output_loss: 0.4007 - department_output_accuracy: 0.8321 - attrition_output_accuracy: 0.8403
Epoch 4/10
35/35 [==============================] - 0s 979us/step - loss: 0.6579 - department_output_loss: 0.2891 - attrition_output_loss: 0.3688 - department_output_accuracy: 0.9029 - attrition_output_accuracy: 0.8494
Epoch 5/10
35/35 [==============================] - 0s 1ms/step - loss: 0.5199 - department_output_loss: 0.1761 - attrition_ou

In [19]:
# Evaluate the model with the testing data
overall_loss, department_loss, attrition_loss, department_accuracy, attrition_accuracy = model.evaluate(X_test_encoded_scaled_df,
                                                                                                        {'department_output': y_test_department, 'attrition_output': y_test_attrition})

12/12 [==============================] - 0s 2ms/step - loss: 0.4182 - department_output_loss: 0.0911 - attrition_output_loss: 0.3271 - department_output_accuracy: 0.9783 - attrition_output_accuracy: 0.8723


In [20]:
# Print the accuracy for both department and attrition
print(f"Department Accuracy: {department_accuracy}")
print(f"Attrition Accuracy: {attrition_accuracy}")

Department Accuracy: 0.97826087474823
Attrition Accuracy: 0.8722826242446899


In [21]:
# Check for imbalances in 'Attrition' column...
attrition_df['Attrition'].value_counts()

No     1233
Yes     237
Name: Attrition, dtype: int64

In [23]:
attrition_df['Department'].value_counts()

Research & Development    961
Sales                     446
Human Resources            63
Name: Department, dtype: int64

# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. Accuracy is not the best metric to use on this data because there are significanly fewer instances of attrition than non-attrition, so the model has a high likely-hood of correctly predicting 'No' for attrition, but low accuracy for predicting 'Yes' for attrition. F1 Score would be a better metric to be able to see how the imbalanced data is affecting the scores. There is a similar, but less concerning, imbalance with the 'Department' column, where the Human Resouce department is represented by less than 5% of the data.

2. Since the 'Attrittion' output is binary, I used the 'Sigmoid' activation function. Since the 'Department' output is multiclass, I used the 'Softmax' activation function.

3. This model could be improved by implementing over-sampling techniques to augment the heavy imbalances in the data, particularly for the 'Attrition' column. Feature engineering could also improve the results if correlated features are removed or transformed to avoid over-weighting correlated relationships. Tinkering with the number of layers, nodes and epochs in the model might also improve the model's evaluation scores. It would also be worthwile to dive deeper into the context of the dataset and the implementation of the model to determine whether correctly identifying specific outcomes, such as false postives, or false negatives, are more important than others.